# Imports

In [280]:
import pandas as pd
import numpy as np
import svgwrite

# Data

In [422]:
vac_df = pd.read_csv('/Users/santa/Projects/vaccine-world-cup/data/data.csv').drop(0)
vac_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10353 entries, 1 to 10353
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   location                             10353 non-null  object 
 1   iso_code                             10353 non-null  object 
 2   date                                 10353 non-null  object 
 3   total_vaccinations                   6527 non-null   object 
 4   people_vaccinated                    5922 non-null   float64
 5   people_fully_vaccinated              4222 non-null   float64
 6   daily_vaccinations_raw               5552 non-null   float64
 7   daily_vaccinations                   10162 non-null  object 
 8   total_vaccinations_per_hundred       6527 non-null   float64
 9   people_vaccinated_per_hundred        5922 non-null   float64
 10  people_fully_vaccinated_per_hundred  4222 non-null   float64
 11  daily_vaccinations_per_milli

In [423]:
vac_df.total_vaccinations = pd.to_numeric(vac_df.total_vaccinations)
vac_df.date = pd.to_datetime(vac_df.date)

In [424]:
vac_df[vac_df.location=='World'].sample(1)

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
10205,World,OWID_WRL,2020-12-20,2780165.0,1267427.0,NaN,1153245.0,384814,0.04,0.02,NaN,49.0


In [425]:
pop_df = pd.read_csv('/Users/santa/Projects/vaccine-world-cup/data/pop_data.csv')
pop_df.head()

,entity,iso_code,year,population
0,Afghanistan,AFG,2020,38928341
1,Albania,ALB,2020,2877800
2,Algeria,DZA,2020,43851043
3,American Samoa,ASM,2020,55197
4,Andorra,AND,2020,77265


In [426]:
df = vac_df[['location','date','iso_code','people_vaccinated','people_vaccinated_per_hundred']].merge(pop_df[['iso_code','population']], on=['iso_code'], how='inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9838 entries, 0 to 9837
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   location                       9838 non-null   object        
 1   date                           9838 non-null   datetime64[ns]
 2   iso_code                       9838 non-null   object        
 3   people_vaccinated              5576 non-null   float64       
 4   people_vaccinated_per_hundred  5576 non-null   float64       
 5   population                     9838 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 538.0+ KB


# SVG

In [375]:
HEIGHT = 800
WIDTH = 200

In [428]:
df[df.location=='India'].people_vaccinated_per_hundred.max()

4.76

In [483]:
dont_draw_list = ['China']
draw_text_finish_line_list = ["World"]

def get_graph_for_location( location, color='black', hor_scale=1.4, ver_scale=2, emoji="🏳"): 
    fill = color

    format_df = df[df.location=='World'].sort_values('date').reset_index()['date'].to_frame()
    location_df = df[df.location==location].set_index('date').people_vaccinated.fillna(method='bfill').to_frame().reset_index()
    m_df = format_df.merge(location_df, on=['date'], how='left').fillna(0)

    vacc_list = [ HEIGHT-(i * ver_scale) for i in ((m_df.people_vaccinated/10000000).values.tolist())]
    ran_list = [ (i * hor_scale) for i in  list(range(0, len(vacc_list)))]
    path_string = ""
    for i,j in zip(ran_list, vacc_list) :
        path_string += "L "
        path_string += (str(i)+" ")
        path_string += (str(j)+" ")
    print(path_string[:20],":::", path_string[-20:])
    d_string_area = 'M 0 '+str(HEIGHT)+ path_string + 'L '+str(ran_list[-1])+' '+str(HEIGHT)+' z'
    d_string_line = 'M 0 '+str(HEIGHT)+ path_string + ' '
    
    area_svg = "<path fill=\"{1}\" stroke-width=0.5 opacity=0.1 d=\"{0}\"/>".format(d_string_area, color)
    line_svg = "<path fill=\"none\" stroke-width=0.5 stroke=\"{1}\" d=\"{0}\"/>".format(d_string_line, color)

    text_svg = '''
    <text 
        x="{0}" y="{1}" 
        font-family="Verdana" 
        text-anchor="start" 
        font-size="4" 
        transform="scale (0.5,1)" 
        fill="{3}">
        {4}&nbsp;{2}&nbsp;
            <tspan
                font-family="Verdana" 
                text-anchor="start" 
                font-size="3" 
                transform="scale (0.5,1)" 
                fill="#eee">
                {5:.1f}%
            </tspan>
    </text>'''.format(
        ran_list[-1]*2 + 2,
        vacc_list[-1],
        location,
        color,
        emoji,
        df[df.location==location].people_vaccinated_per_hundred.max()
    )

    finish_line_loc = HEIGHT - df[df.location==location].population.values[0]/10000000
    finish_line_svg = '''
    <path 
        fill="none" stroke="{color}" 
        stroke-width="0.5" 
        stroke-dasharray="1,0.5" 
        d="M0 {value} l200 0">
    </path>'''.format( 
        value =  finish_line_loc,
        color = color)

    if location in draw_text_finish_line_list:
        finish_line_svg +='''
        <text x="375" y="{value}" 
        font-family="Verdana" 
        text-anchor="end" font-size="5" transform="scale (0.5,1)" fill="white">World Finish Line</text>'''.format(value=finish_line_loc-1)

    print(location, dont_draw_list)
    if location in dont_draw_list:
        ret_string = "\t {}".format(finish_line_svg)
    else :
        ret_string =  "\t {0}\n\t {1}\n\t {2}\n\t {3}".format(area_svg, line_svg,text_svg, finish_line_svg)

    return ret_string

def group_location (locations = [], colors = [], emoji=[]):
    ret_string = ""

    for l, c, e in zip(locations, colors, emoji):
        ret_string += (get_graph_for_location(location=l, color=c, emoji=e))

    return ret_string

def grid_pattern():

    ret_string = '''
        <defs>
            <pattern 
                id="pattern1" 
                patternUnits="userSpaceOnUse" 
                x="0" y="0" width="10" height="10" 
                viewBox="0 0 4 4">
                <path 
                    d="M 0 0 L 0 4 L 4 4 L 4 0 Z" 
                    fill="none" 
                    stroke-width="0.05" 
                    stroke="#575757"></path>
            </pattern>
  
        </defs>
        <g>
            <rect x="0" y="0" 
                  width="100%" 
                  height="100%" 
                  fill="url(#pattern1)" />
        </g>
    '''
    return ret_string

def background():
    return '''
        <defs xmlns="http://www.w3.org/2000/svg">
					<linearGradient xmlns="http://www.w3.org/2000/svg" id="gradient-fill" x1="0" y1="0" x2="800" y2="0" gradientUnits="userSpaceOnUse">				
							<stop offset="0" stop-color="#060d20"/>
                            <stop offset="0.14285714285714285" stop-color="#091228"/>
                            <stop offset="0.2857142857142857" stop-color="#091630"/>
                            <stop offset="0.42857142857142855" stop-color="#091a38"/>
                            <stop offset="0.5714285714285714" stop-color="#0a1d40"/>
                            <stop offset="0.7142857142857142" stop-color="#0b2149"/>
                            <stop offset="0.8571428571428571" stop-color="#0c2451"/>
                            <stop offset="1" stop-color="#0e285a"/>
					</linearGradient>
				</defs>

    <rect fill="url(#gradient-fill)" width="200" height="800"/>
    '''

html_string = '''
<html style="width:100%;height:100%;">
<body style="width:100%;height:100%;margin:0;">
<div style="height: 400%; width: 100%; border:2px solid #000; overflow: scroll; position: absolute;" >
  <svg height="100%" width="100%" viewBox="0 0 200 800"  position="absolute" preserveAspectRatio="none" >
    <path fill="none" stroke="black" stroke-width="4" stroke-dasharray="5,5" d="M5 20 l215 0" />
{background}
{grid_pattern}
{group_location}
 </svg>
</div>

<div style="height: 70%;width: 35%;border:2px;top:15%;left: 3%;background-color: #fff;position:relative;">
<h1 style="font-size: 50px;" "> 🏆 Vaccine World Cup</h1>
</div>

</body>
</html>
'''.format(
    group_location = group_location(
    locations=['World','India', 'United States','European Union', 'Africa','South America', 'China'],
    colors=['#cccccc','#FF9933', '#2D89FF' ,'#0055FF','#00c918','#f8ff33',"#DE2910"],
    emoji=['🌍','🇮🇳','🇺🇸','🇪🇺','🇺🇳','🇺🇳','🇨🇳']),

    grid_pattern = grid_pattern(),

    background = background()
    
    )


f = open('/Users/santa/Projects/vaccine-world-cup/index.html','w')
f.write(html_string)
f.close()

L 0.0 799.982707 L 1 ::: 99999998 727.261023 
World ['China']
L 0.0 800.0 L 1.4 80 ::: 99999998 786.852106 
India ['China']
L 0.0 800.0 L 1.4 80 ::: 9999998 779.1573044 
United States ['China']
L 0.0 800.0 L 1.4 80 ::: 9999998 788.8784882 
European Union ['China']
L 0.0 800.0 L 1.4 80 ::: 9999998 798.4896736 
Africa ['China']
L 0.0 800.0 L 1.4 80 ::: 99999998 793.879511 
South America ['China']
L 0.0 800.0 L 1.4 80 ::: 9999999999998 800.0 
China ['China']


In [400]:
df.location.unique()

array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla', 'Antigua and Barbuda', 'Argentina', 'Asia',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bermuda', 'Bhutan', 'Bolivia', 'Brazil', 'Brunei', 'Bulgaria',
       'Cambodia', 'Canada', 'Cape Verde', 'Cayman Islands', 'Chile',
       'China', 'Colombia', 'Costa Rica', "Cote d'Ivoire", 'Croatia',
       'Cyprus', 'Czechia', 'Denmark', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Estonia',
       'Europe', 'European Union', 'Faeroe Islands', 'Falkland Islands',
       'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada',
       'Guatemala', 'Guernsey', 'Guinea', 'Guyana', 'Honduras',
       'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran',
       'Iraq', 'Ireland', 'Isle